In [ ]:
#Import the required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import warnings
from sklearn.model_selection import KFold 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from matplotlib.pyplot import figure
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
warnings.filterwarnings("ignore", category = RuntimeWarning)
warnings.filterwarnings('ignore', 'Solver terminated early.*')

#-------------------------------------------------------------------------------------------------------------
#Loading the Data:
#Loading train data
train_data=pd.read_excel(open(r"C:\Users\simon\Downloads\Data_Mahbobeh.xlsx", 'rb'),sheet_name='Train') 
#train_data.head()
#Loading HC test data
HC_test_data=pd.read_excel(open(r"C:\Users\simon\Downloads\Data_Mahbobeh.xlsx", 'rb'),sheet_name='Test_1') 
#HC_test_data.head()
#Loading MCI test data
MCI_test_data=pd.read_excel(open(r"C:\Users\simon\Downloads\Data_Mahbobeh.xlsx", 'rb'),sheet_name='Test_2') 
#MCI_test_data.head()
#loading AD Test Data
AD_test_data=pd.read_excel(open(r"C:\Users\simon\Downloads\Data_Mahbobeh.xlsx", 'rb'),sheet_name='Test_3') 
#AD_test_data.head()

#-------------------------------------------------------------------------------------------------------------
#Label and Features Definition
#Training Set , 675 HC
#HC age Train: label
#HC Data Train : features
HC_DATA_Train = train_data.iloc[:, 1:-1].values
HC_Age_Train= train_data.iloc[:, 0].values

# Test sets
#Test 1: independent 75 HC
#HC age Test: label
#HC Data Test : features
HC_DATA_Test= HC_test_data.iloc[:, 1:-1].values
HC_Age_Test = HC_test_data.iloc[:, 0].values

#Test 2 : MCI
#MCI data: features
#MCI age : label
MCI_DATA= MCI_test_data.iloc[:, 1:-1].values
MCI_Age = MCI_test_data.iloc[:, 0].values

#Test 3 : AD
#AD_Data is the features
#AD_Age is the label
AD_DATA= AD_test_data.iloc[:, 1:-1].values
AD_Age = AD_test_data.iloc[:, 0].values


#Training Set , 675 HC
#HC age Train: label
#HC Data Train : features
HC_DATA_Train = train_data.iloc[:, 1:-1].values
HC_Age_Train= train_data.iloc[:, 0].values
print('Training Features Shape:', HC_DATA_Train.shape)
print('Training Labels Shape:', HC_Age_Train.shape)

#-------------------------------------------------------------------------------------------------------------
Simulation_State=1  # 1: Without bias correction  2: Cole's method   3: Proposed Method

#-------------------------------------------------------------------------------------------------------------
#10-fold Cross Validation On Train Set

# call the models
model_1 = LinearRegression() 

PredictTest_Before=np.zeros((len(HC_DATA_Train),1))
X=HC_DATA_Train
y=HC_Age_Train

k = 10  # Number of folds 
kf = KFold(n_splits=k, random_state=None)

for train_index , test_index in kf.split(X):
    X_train , X_test = X[train_index,:],X[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]  
    model_1.fit(X_train,y_train)
    pred_values1 = model_1.predict(X_test)
    pred_values1.shape=(len(pred_values1),1)
    PredictTest_Before[test_index]=pred_values1
    

    
print('HC_Age_Train Shape:', HC_Age_Train.shape)
print('PredictTest_Before Shape:', PredictTest_Before.shape)
#------------------------------------------------------------------------------------------------------------

if Simulation_State==2:
    a1,b1= np.polyfit(HC_Age_Train, PredictTest_Before, 1)
elif Simulation_State==3:
    a2,b2=np.polyfit(HC_Age_Train, PredictTest_Before -HC_Age_Train, 1)


PredictTest=list()

if Simulation_State==1:
    PredictTest=PredictTest_Before
    
elif Simulation_State==2:
    for age in PredictTest_Before:
        PredictTest.append((age-b1)/a1)
    PredictTest=np.asarray(PredictTest)
    
elif Simulation_State==3:
    Offset=list()
    for age in HC_Age_Train:
        age=(age*a2)+b2
        Offset.append(age)
    PredictTest=PredictTest_Before-Offset

#------------------------------------------------------------------------------------------------------------    
#Metrics(Evaluating the model performance)
r_train = np.corrcoef(HC_Age_Train, PredictTest)
R2_Train= np.mean(r_train * r_train)
print ('R Squared_Train =',R2_Train)
print ('MAE_Train =',mean_absolute_error(HC_Age_Train, PredictTest))
print ('RMSE_Train =',math.sqrt(mean_squared_error(HC_Age_Train, PredictTest)))

Delta_age_train=PredictTest -HC_Age_Train
MEANHCs=np.mean(Delta_age_train)
print("Mean Delta Brain Age for Training set: ", MEANHCs)

#------------------------------------------------------------------------------------------------------------    
#Plots
#Predicted Brain age Vs Real age (Train Data)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)

plt.subplot(2, 2, 1)
plt.scatter(HC_Age_Train, PredictTest, c='crimson')
p1 = max(max(PredictTest), max(HC_Age_Train))
p2 = min(min(PredictTest), min(HC_Age_Train))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Real Brain age_Training', fontsize=12)
plt.ylabel('Predicted Brain age_Training', fontsize=12)
plt.title('Predicted Age Vs. Real Age for Train', fontsize=14)
#plt.axis('equal')
plt.tight_layout()
plt.show()

#Delta brain age Vs Real Age(Train Data)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
PredictTest.shape=(675,)
Delta_age_train=PredictTest-HC_Age_Train
plt.subplot(2, 2, 1)
plt.scatter(HC_Age_Train, Delta_age_train, c='crimson')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(HC_Age_Train, Delta_age_train, 1)
#add linear regression line to scatterplot 
plt.plot(HC_Age_Train, m*HC_Age_Train+b)
plt.xlabel('Real Brain Age(year)', fontsize=15)
plt.ylabel('Delta Brain Age(year)', fontsize=15)
plt.title('Delta Brain Age Vs. Real Age for Train Set', fontsize=18)
#plt.axis('equal')
plt.tight_layout()
plt.show()
#------------------------------------------------------------------------------------------------------------  

############################################## . ON INDEPENDENT TEST SETS #######################################
##############################INDEPENDENT HC###########################################
#Global Model:
Mdl_onTest_HC = LinearRegression().fit(HC_DATA_Train, HC_Age_Train)
PredictTest_HC_Before = Mdl_onTest_HC.predict(HC_DATA_Test)

#States
PredictTest_HC=list()

if Simulation_State==1:
    PredictTest_HC=PredictTest_HC_Before
    
elif Simulation_State==2:
    for age in PredictTest_HC_Before:
        PredictTest_HC.append((age-b1)/a1)
    PredictTest_HC=np.asarray(PredictTest_HC)
    
elif Simulation_State==3:
    Offset=list()
    for age in HC_Age_Test:
        age=(age*a2)+b2
        Offset.append(age)
    PredictTest_HC=PredictTest_HC_Before-Offset

#Metrics for HC independent:
r_hc = np.corrcoef(HC_Age_Test, PredictTest_HC)
R2_HC= np.mean(r_hc * r_hc)
print ('R Squared_HC =',R2_HC)
print ('MAE_HC =',mean_absolute_error(HC_Age_Test, PredictTest_HC))
print ('RMSE_HC =',math.sqrt(mean_squared_error(HC_Age_Test, PredictTest_HC)))
Mean_HC_Final=np.mean((PredictTest_HC-HC_Age_Test))
print("Mean Delta Brain Age for HC independent Test set: ", Mean_HC_Final)
#------------------------------------------------------------------------------------------------------------ 

#plot predicted Vs Real age(Independent HC)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 2)
plt.scatter(HC_Age_Test, PredictTest_HC, c='crimson')
p1 = max(max(PredictTest_HC), max(HC_Age_Test))
p2 = min(min(PredictTest_HC), min(HC_Age_Test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Real Brain age_HC independent Test Set', fontsize=12)
plt.ylabel('Predicted Brain age_HC independet Test Set', fontsize=12)
plt.title('Predicted Age Vs. Real Age for HC Test Set', fontsize=14)
plt.tight_layout()
#plt.axis('equal')
plt.show()

#Delta brain age vs real age(Independent HC)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 2)
Delta_brain_Age_HC=PredictTest_HC-HC_Age_Test
plt.scatter(HC_Age_Test, Delta_brain_Age_HC, c='crimson')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(HC_Age_Test, Delta_brain_Age_HC, 1)
#add linear regression line to scatterplot 
plt.plot(HC_Age_Test, m*HC_Age_Test+b)
plt.xlabel('Real Brain Age_HC independent Test Set(year)', fontsize=15)
plt.ylabel('Delta Brain Age_HC independent Test Set(year)', fontsize=15)
plt.title('Delta Brain Age Vs. Real Age for HC independent Set', fontsize=18)
plt.tight_layout()
#plt.axis('equal')
plt.show()

#------------------------------------------------------------------------------------------------------------ 
######################################TEST ON MCI##################################################################
Mdl_onTest_MCI=LinearRegression().fit(HC_DATA_Train, HC_Age_Train)
PredictTest_MCI_Before=Mdl_onTest_MCI.predict(MCI_DATA)

#States
PredictTest_MCI=list()

if Simulation_State==1:
    PredictTest_MCI=PredictTest_MCI_Before
    
elif Simulation_State==2:
    for age in PredictTest_MCI_Before:
        PredictTest_MCI.append((age-b1)/a1)
    PredictTest_MCI=np.asarray(PredictTest_MCI)
    
elif Simulation_State==3:
    Offset=list()
    for age in MCI_Age:
        age=(age*a2)+b2
        Offset.append(age)
    PredictTest_MCI=PredictTest_MCI_Before-Offset

#Metrics for HC independent:
r_MCI = np.corrcoef(MCI_Age, PredictTest_MCI)
R2_MCI= np.mean(r_MCI * r_MCI)
print ('R Squared_MCI =',R2_MCI)
print ('MAE_MCI =',mean_absolute_error(MCI_Age, PredictTest_MCI))
print ('RMSE_MCI =',math.sqrt(mean_squared_error(MCI_Age, PredictTest_MCI)))
Mean_MCI_Final=np.mean((PredictTest_MCI-MCI_Age))
print("Mean Delta Brain Age for MCI: ", Mean_MCI_Final)

#----------------------------------------Plots---------------------------------------
#plot predicted Vs Real age(MCI)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 3)
plt.scatter(MCI_Age, PredictTest_MCI, c='crimson')
p1 = max(max(PredictTest_MCI), max(MCI_Age))
p2 = min(min(PredictTest_MCI), min(MCI_Age))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Real Brain age', fontsize=12)
plt.ylabel('Predicted Brain age', fontsize=12)
plt.title('Predicted Age Vs. Real Age for MCI Test Set', fontsize=14)
plt.tight_layout()
#plt.axis('equal')
plt.show()

#------------------------------------------------------------------
#Delta brain age vs real age(MCI)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 3)
Delta_brain_Age_MCI=PredictTest_MCI-MCI_Age
plt.scatter(HC_Age_Test, Delta_brain_Age_HC, c='crimson')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(MCI_Age, Delta_brain_Age_MCI, 1)
#add linear regression line to scatterplot 
plt.plot(MCI_Age, m*MCI_Age+b)
plt.xlabel('Real Brain Age_MCI (year)', fontsize=15)
plt.ylabel('Delta Brain Age_MCI(year)', fontsize=15)
plt.title('Delta Brain Age Vs. Real Age for MCI Set', fontsize=18)
plt.tight_layout()
#plt.axis('equal')
plt.show()


########################################## TEST ON AD-###############################################################
Mdl_onTest_AD =LinearRegression().fit(HC_DATA_Train, HC_Age_Train)
PredictTest_AD_Before=Mdl_onTest_AD.predict(AD_DATA)

#States
PredictTest_AD=list()

if Simulation_State==1:
    PredictTest_AD=PredictTest_AD_Before
    
elif Simulation_State==2:
    for age in PredictTest_AD_Before:
        PredictTest_AD.append((age-b1)/a1)
    PredictTest_AD=np.asarray(PredictTest_AD)
    
elif Simulation_State==3:
    Offset=list()
    for age in AD_Age:
        age=(age*a2)+b2
        Offset.append(age)
    PredictTest_AD=PredictTest_AD_Before-Offset

#Metrics for HC independent:
r_AD = np.corrcoef(AD_Age, PredictTest_AD)
R2_AD= np.mean(r_AD * r_AD)
print ('R Squared_AD =',R2_AD)
print ('MAE_AD =',mean_absolute_error(AD_Age, PredictTest_AD))
print ('RMSE_AD =',math.sqrt(mean_squared_error(AD_Age, PredictTest_AD)))
Mean_AD_Final=np.mean((PredictTest_AD-AD_Age))
print("Mean Delta Brain Age for AD: ", Mean_AD_Final)

#----------------------------------------Plots---------------------------------------
#plot predicted Vs Real age(AD)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 4)
plt.scatter(AD_Age, PredictTest_AD, c='crimson')
p1 = max(max(PredictTest_AD), max(AD_Age))
p2 = min(min(PredictTest_AD), min(AD_Age))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Real Brain age_ AD', fontsize=12)
plt.ylabel('Predicted Brain age_AD', fontsize=12)
plt.title('Predicted Age Vs. Real Age for AD Test Set', fontsize=14)
plt.tight_layout()
#plt.axis('equal')
plt.show()
#------------------------------------------------------------------
#Delta brain age vs real age(AD)
ig = plt.figure()
figure(figsize=(15,10), dpi=80)
plt.subplot(2, 2, 4)
Delta_brain_Age_AD=PredictTest_AD- AD_Age
plt.scatter(AD_Age, Delta_brain_Age_AD, c='crimson')
#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(AD_Age, Delta_brain_Age_AD, 1)
#add linear regression line to scatterplot 
plt.plot(AD_Age, m*AD_Age+b)
plt.xlabel('Real Brain Age_AD(year)', fontsize=15)
plt.ylabel('Delta Brain Age_AD(year)', fontsize=15)
plt.title('Delta Brain Age Vs. Real Age for AD Set', fontsize=18)
#plt.axis('equal')
#set the spacing between subplots
plt.tight_layout()
plt.show()
#------------------------------------------------------------------------------------------------------------ 
#---------------------------Plot the table of results---------------------------------------

print(" Table of Results")
# import module
from tabulate import tabulate

# assign data
mydata = [
    ["Train Data", R2_Train,
     mean_absolute_error(HC_Age_Train, PredictTest),
     math.sqrt(mean_squared_error(HC_Age_Train, PredictTest)), MEANHCs],
    ["Healthy Independent Test",R2_HC,
     mean_absolute_error(HC_Age_Test, PredictTest_HC),
     math.sqrt(mean_squared_error(HC_Age_Test, PredictTest_HC)), Mean_HC_Final],
    ["MCI Test", R2_MCI,
     mean_absolute_error(MCI_Age, PredictTest_MCI),
     math.sqrt(mean_squared_error(MCI_Age, PredictTest_MCI)), Mean_MCI_Final],
      ["AD Test", R2_AD, mean_absolute_error(AD_Age, PredictTest_AD),
       math.sqrt(mean_squared_error(AD_Age, PredictTest_AD)), Mean_AD_Final]
]
 
# create header
head = ["Dataset", "R_ Squared", "MAE", "RMSE", "Delta Brain Age"]
 
# display table
print(tabulate(mydata, headers=head, tablefmt="grid"))

